# 生成器 generator
一种特殊的迭代器 iterator，在需要时动态生成值，实现“惰性计算”

## 列表推导式与生成器表达式
- 大型数据集或流数据时，生成器表达式更优，节省内存
  - 如处理文件字节长度 sum(x for x in (len(line) for line in open(filename)))
- 小型数据集，列表推导式更快，可一次性生成所有值

In [ ]:
## 1. 列表推导式 <class 'list'>
lst_generator = [0 for _ in range(10)] # 10 个元素为0 的列表
print(f'lst_generator: {lst_generator}, 其类型为{type(lst_generator)}')

## 2. 生成器表达式 <class 'generator'>，与列表推导式类似，但返回的是生成器对象而不是列表
generator_expression = (0 for _ in range(10))
print(f'generator_expression: {generator_expression}, 其类型为{type(generator_expression)}')

print(next(generator_expression))
print(next(generator_expression))
# 注意 next 次数不可超过 generator_expression 的长度，否则抛错 StopIteration

## 3. 生成器函数的使用
# 使用 yield 返回值，不是 return
# 使用生成器函数会返回一个迭代器，可以使用 next() 函数获取值

def my_generator():
    for i in range(10):
        yield i

gen = my_generator()
print(next(gen)) # 0
print(next(gen)) # 1

# 生成器双向通信：不仅可以向调用者返回值，还可以从调用者接收值（PS：进阶：还支持 throw 和 close 方法）
def send_generator_for_plus():
    x = yield
    while True:
        x = yield x + x

sg = send_generator_for_plus()
next(sg)
print(sg.send(10))
print(sg.send(3))





lst_generator: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 其类型为<class 'list'>
generator_expression: <generator object <genexpr> at 0x10f463510>, 其类型为<class 'generator'>
0
0
0
1
20
6


# Python 管理内存
- objgraph
- memory_profiler
- tracemalloc